In [1]:
from aoc2023 import day1
from aoc2023 import maps
import numpy as np
import functools

In [141]:
data = day1.read_data_string("input.txt")
# data = day1.read_data_string("input_sample.txt")
lines = day1.split_data(data) 
paragraphs = [day1.split_data_p(paragraph) for paragraph in day1.split_data_paragraphs(data)]

In [142]:
def parse_connections(lines): 
    connections = {}
    flip_flops = set()
    conjunctions = set()
    for l in lines: 
        node, cons = l.split(" -> ")
        if '%' in node: 
            node = node.replace('%', '')
            flip_flops.add(node)
        elif '&' in node: 
            node = node.replace('&', '')
            conjunctions.add(node)
        else : 
            print("broadcast")
        
        cons = cons.split(', ')
        connections[node] = cons
    
    return connections, flip_flops, conjunctions


In [202]:
cons, flips, conjunctions = parse_connections(lines)

broadcast


In [255]:
flips_state = {f:"low" for f in flips}
conjunctions_state = {c:{f:"low" for f in flips if c in cons[f]} for c in conjunctions}


In [256]:

from queue import Queue


In [257]:
conjunctions_state["rm"]

{}

In [258]:
count_h = 0 
count_l = 0

In [259]:

list_x = []
for i in range(10000):
    count_x = 0
    actions = Queue()
    actions.put(("low", "broadcaster", "button"))

    def process_flip(flip, signal): 
        if signal == "high": 
            pass 
        else:
            if flips_state[flip] == "high": 
                flips_state[flip] = "low"
                for c in cons[flip]: 
                    actions.put(("low", c, flip))
            else :
                flips_state[flip] = "high"
                for c in cons[flip]: 
                    actions.put(("high", c, flip))


    def process_conjunction(conj, signal, input_node): 
        c_state = conjunctions_state[conj]

        if signal == "high": 
            c_state[input_node] = "high"
        else:
            c_state[input_node] = "low"
        
        output = "low"
        for _, value in c_state.items(): 
            if value == "low": 
                output = "high"
                break

        for c in cons[conj]: 
            actions.put((output, c, conj))

    found = False
    while not actions.empty():
        a, node, previous = actions.get()
        # print(a, node)
        if a == "low": 
            count_l += 1
        if a == "high": 
            count_h += 1
        if node in flips: 
            process_flip(node, a)
        if node in conjunctions: 
            process_conjunction(node, a, previous)
        if node == "broadcaster": 
            for c in cons[node]: 
                actions.put((a, c, node))
        
        if node == "bb":
            if a == "low": 
                found = True

    if found: 
        print(i+1)
        break
        # print(list(actions.queue))

3907


In [239]:
#!/bin/python3

import sys
import math
from typing import List, Tuple

sys.setrecursionlimit(100000)
FILE = "input.txt"


def read_lines_to_list() -> List[str]:
    lines: List[str] = []
    with open(FILE, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            split = line.split(" -> ")

            name = split[0]
            flip_flop = name.startswith("%")
            conjunction = name.startswith("&")
            target = split[1].split(", ")

            if flip_flop:
                state = False
                name = split[0][1:]
            elif conjunction:
                state = {}
                name = split[0][1:]
            else:
                state = None

            val = (name, [target, flip_flop, conjunction, state])

            lines.append(val)

    return lines


def part_one():
    lines = read_lines_to_list()
    mappings = dict((a, b) for (a, b) in lines)

    # For any conjunction modules we must initialize inputs...
    for k, v in mappings.items():
        if v[2]:
            for a, b in mappings.items():
                if k in b[0]:
                    v[3][a] = False

    low = 0
    high = 0
    for _ in range(1000):
        queue = [("broadcaster", 0, None)]
        while queue:
            (curr, signal, input) = queue.pop(0)

            if signal:
                high += 1
            else:
                low += 1

            if curr not in mappings:
                continue

            [targets, is_ff, is_con, state] = mappings[curr]

            if is_ff:
                if not signal:
                    if state:
                        mappings[curr][3] = False
                        new_signal = 0
                    else:
                        mappings[curr][3] = True
                        new_signal = 1

                    for target in targets:
                        queue.append((target, new_signal, curr))
            elif is_con:
                state[input] = bool(signal)
                if all(state.values()):
                    new_signal = 0
                else:
                    new_signal = 1
                for target in targets:
                    queue.append((target, new_signal, curr))
            else:
                for target in targets:
                    queue.append((target, signal, curr))

    answer = low * high
    print(f"Part 1: {answer}")


def part_two():
    lines = read_lines_to_list()
    mappings = dict((a, b) for (a, b) in lines)

    for k, v in mappings.items():
        if v[2]:
            for a, b in mappings.items():
                if k in b[0]:
                    v[3][a] = False

    # A little bit of hardcoded trickery for my input... I specifically needed xr, fv, kk, and vt to all be low.
    lowest_parents = {
        "qd": None,
        "dp": None,
        "dh": None,
        "bb": None,
    }

    curr_cycle = 0
    answer = 0
    while True:
        if all(val is not None for val in lowest_parents.values()):
            print(lowest_parents)
            answer = math.lcm(*list(lowest_parents.values()))
            break

        curr_cycle += 1
        queue = [("broadcaster", 0, None)]
        while queue:
            (curr, signal, input) = queue.pop(0)

            if curr in lowest_parents and not signal:
                lowest_parents[curr] = curr_cycle

            # Ideally this happens but it won't.
            if curr == "rx" and not signal:
                break

            if curr not in mappings:
                continue

            [targets, is_ff, is_con, state] = mappings[curr]

            if is_ff:
                if not signal:
                    if state:
                        mappings[curr][3] = False
                        new_signal = 0
                    else:
                        mappings[curr][3] = True
                        new_signal = 1

                    for target in targets:
                        queue.append((target, new_signal, curr))
            elif is_con:
                state[input] = bool(signal)
                if all(state.values()):
                    new_signal = 0
                else:
                    new_signal = 1
                for target in targets:
                    queue.append((target, new_signal, curr))
            else:
                for target in targets:
                    queue.append((target, signal, curr))

    print(f"Part 2: {answer}")


part_one()
part_two()


Part 1: 800830848
{'qd': 4001, 'dp': 4027, 'dh': 3877, 'bb': 3907}
Part 2: 244055946148853
